In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import seaborn as sb
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
import warnings

# Suppress specific warning
warnings.filterwarnings("ignore", category=FutureWarning)

# Suppress all warnings
warnings.filterwarnings("ignore")


In [3]:
file_path= r"C:\Users\subha\Desktop\AgriSage\Crop_Recommendation.csv"
raw_data= pd.read_csv (file_path)

In [4]:
features= raw_data.columns[:-1]
features

Index(['Nitrogen', 'Phosphorus', 'Potassium', 'Temperature', 'Humidity',
       'pH_Value', 'Rainfall'],
      dtype='object')

In [5]:
target = 'Crop'

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import QuantileTransformer


def transform_data(df, target, num_features):
    # Ensure num_features is a list
    if isinstance(num_features, pd.Index):
        num_features = num_features.tolist()

    # Encoding target
    lbl_encoder = LabelEncoder()
    df[target+'_Encoded'] = lbl_encoder.fit_transform(df[target])
    
    # Assigning features and labels
    x = df.drop([target, target+'_Encoded'], axis=1)
    y = df[target+'_Encoded']
    
    # Splitting the dataset into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=101)
    
    # Accessing the encoded classes
    encoded_classes = lbl_encoder.classes_
    # Printing the mapping (index corresponds to encoded value, value is the original label)
    for i, label in enumerate(encoded_classes):
        print(f"Encoded Value: {i}, Original Label: {label}")    
    
    # Standardization and Encoding
    # Define transformers for different column types
    std_scaler = StandardScaler()
    quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)

    # Combine transformers for specific columns
    preprocessor = ColumnTransformer([
        ("num", std_scaler, num_features),
        ("num_trns", quantile_transformer, num_features)
    ])
     # Fit transformers on training data only
    preprocessor.fit(x_train)

    # Transform train and test data using fitted transformers
    x_train_transformed = preprocessor.transform(x_train)
    x_test_transformed = preprocessor.transform(x_test)

    # Calculate statistics for numerical features before and after transformation
    stats = {}
    for i, feature in enumerate(num_features):
        stats[feature] = {
            'mean_before': x_train[feature].mean(),
            'std_before': x_train[feature].std(),
            'mean_after': x_train_transformed[:, i].mean(),
            'std_after': x_train_transformed[:, i].std()
        }
    
    return x_train_transformed, x_test_transformed, y_train, y_test, stats

In [21]:
x_train, x_test, y_train, y_test, stats = transform_data(raw_data, target, features)

Encoded Value: 0, Original Label: Apple
Encoded Value: 1, Original Label: Banana
Encoded Value: 2, Original Label: Blackgram
Encoded Value: 3, Original Label: ChickPea
Encoded Value: 4, Original Label: Coconut
Encoded Value: 5, Original Label: Coffee
Encoded Value: 6, Original Label: Cotton
Encoded Value: 7, Original Label: Grapes
Encoded Value: 8, Original Label: Jute
Encoded Value: 9, Original Label: KidneyBeans
Encoded Value: 10, Original Label: Lentil
Encoded Value: 11, Original Label: Maize
Encoded Value: 12, Original Label: Mango
Encoded Value: 13, Original Label: MothBeans
Encoded Value: 14, Original Label: MungBean
Encoded Value: 15, Original Label: Muskmelon
Encoded Value: 16, Original Label: Orange
Encoded Value: 17, Original Label: Papaya
Encoded Value: 18, Original Label: PigeonPeas
Encoded Value: 19, Original Label: Pomegranate
Encoded Value: 20, Original Label: Rice
Encoded Value: 21, Original Label: Watermelon


##model

In [22]:
from sklearn.model_selection import StratifiedKFold, cross_validate


def model_comparison(x, y, models, stats):
    names = []
    scoring = ['accuracy']
    
    # Create a dataframe to store the different metric values for each algorithm
    df_results = pd.DataFrame(columns=['Algorithm', 'Acc Mean', 'Acc STD'])
    results_acc = [] # List of accuracy scores for each fold of each algorithm
    
    for name, model in models:
        names.append(name)
        kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=101)
        result = cross_validate(model, x, y, cv=kfold, scoring=scoring)
        # Mean and standard deviation of Accuracy scores for the algorithm
        acc_mean = result['test_accuracy'].mean()
        acc_std = result['test_accuracy'].std()
        
        # Create the row of the results
        df_result_row = {'Algorithm': name, 'Acc Mean': acc_mean, 'Acc STD': acc_std}
        # Add the row to the results data frame
        df_results = pd.concat([df_results, pd.DataFrame([df_result_row])], ignore_index=True)
        
        results_acc.append(result['test_accuracy'])

    # Add the means and standard deviations of the encoded variables to the results dataframe
    for feature, stat in stats.items():
        df_results[f'{feature} Mean Before'] = stat['mean_before']
        df_results[f'{feature} STD Before'] = stat['std_before']
        df_results[f'{feature} Mean After'] = stat['mean_after']
        df_results[f'{feature} STD After'] = stat['std_after']
        
    df_results = df_results.set_index('Algorithm')
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    # Display the mean and standard deviation of all metrics for all algorithms
    print(df_results)

In [9]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier


ens_models = []
ens_models.append(('RFC', RandomForestClassifier()))
ens_models.append(('ABC', AdaBoostClassifier()))
ens_models.append(('GBC', GradientBoostingClassifier()))

In [23]:
model_comparison(x_train, y_train, ens_models,stats)

           Acc Mean  Acc STD  Nitrogen Mean Before  Nitrogen STD Before  \
Algorithm                                                                 
RFC           0.993    0.005                50.464               36.970   
ABC           0.177    0.014                50.464               36.970   
GBC           0.988    0.009                50.464               36.970   

           Nitrogen Mean After  Nitrogen STD After  Phosphorus Mean Before  \
Algorithm                                                                    
RFC                      0.000               1.000                  53.398   
ABC                      0.000               1.000                  53.398   
GBC                      0.000               1.000                  53.398   

           Phosphorus STD Before  Phosphorus Mean After  Phosphorus STD After  \
Algorithm                                                                       
RFC                       33.133                 -0.000                

ensemble model vs non linear models.
DecisionTreeClassifier()
KNeighborsClassifier()
GaussianNB()
XGBClassifier()